# Clustering Crypto

In [69]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from urllib.request import Request, urlopen
import os
import json

In [70]:
def read_json(url):
    request = Request(url)
    response = urlopen(request)
    data = response.read()
    url2 = json.loads(data)
    return url2

### Fetching Cryptocurrency Data

In [71]:
#Request the raw data from CryptoCompare, and store it as a var
url = f"https://min-api.cryptocompare.com/data/all/coinlist"
request = Request(url)
response = urlopen(request)
data = response.read()

In [74]:
#Convert and store the raw data as json data in a var
json_data = json.loads(data)

In [75]:
#Check the Data
json_data['Data']

{'42': {'Id': '4321',
  'Url': '/coins/42/overview',
  'ImageUrl': '/media/35650717/42.jpg',
  'ContentCreatedOn': 1427211129,
  'Name': '42',
  'Symbol': '42',
  'CoinName': '42 Coin',
  'FullName': '42 Coin (42)',
  'Algorithm': 'Scrypt',
  'ProofType': 'PoW/PoS',
  'FullyPremined': '0',
  'TotalCoinSupply': '42',
  'BuiltOn': 'N/A',
  'SmartContractAddress': 'N/A',
  'PreMinedValue': 'N/A',
  'TotalCoinsFreeFloat': 'N/A',
  'SortOrder': '34',
  'Sponsored': False,
  'Taxonomy': {'Access': '',
   'FCA': '',
   'FINMA': '',
   'Industry': '',
   'CollateralizedAsset': '',
   'CollateralizedAssetType': '',
   'CollateralType': '',
   'CollateralInfo': ''},
  'IsTrading': True,
  'TotalCoinsMined': 41.99995317,
  'BlockNumber': 147668,
  'NetHashesPerSecond': 0,
  'BlockReward': 0,
  'BlockTime': 0},
 '300': {'Id': '749869',
  'Url': '/coins/300/overview',
  'ImageUrl': '/media/27010595/300.png',
  'ContentCreatedOn': 1517935016,
  'Name': '300',
  'Symbol': '300',
  'CoinName': '300 to

In [76]:
# Creating a DataFrame
df = pd.DataFrame(json_data['Data']).T
df.head()


,Algorithm,BlockNumber,BlockReward,BlockTime,BuiltOn,CoinName,ContentCreatedOn,FullName,FullyPremined,Id,...,ProofType,SmartContractAddress,SortOrder,Sponsored,Symbol,Taxonomy,TotalCoinSupply,TotalCoinsFreeFloat,TotalCoinsMined,Url
42,Scrypt,147668,0,0,N/A,42 Coin,1427211129,42 Coin (42),0,4321,...,PoW/PoS,N/A,34,False,42,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",42,N/A,42,/coins/42/overview
300,N/A,0,0,0,7605,300 token,1517935016,300 token (300),0,749869,...,N/A,0xaec98a708810414878c3bcdf46aad31ded4a4557,2212,False,300,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",300,N/A,300,/coins/300/overview
365,X11,NaN,NaN,NaN,N/A,365Coin,1480032918,365Coin (365),0,33639,...,PoW/PoS,N/A,916,False,365,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",2300000000,N/A,NaN,/coins/365/overview
404,Scrypt,236517,0,60,N/A,404Coin,1466100361,404Coin (404),0,21227,...,PoW/PoS,N/A,602,False,404,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",532000000,N/A,0,/coins/404/overview
433,N/A,8837713,2,0,7605,433 Token,1541597321,433 Token (433),0,926547,...,N/A,0x738505a5f31bf72e0b70298bca81150eb1b7c751,3505,False,433,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",1000000000,N/A,1.08349e+08,/coins/433/overview


### Data Preprocessing

In [77]:
# Remove unnecessary columns
trimmed_df = df.drop(columns=['TotalCoinsFreeFloat', 'Url', 'Taxonomy', 'SortOrder', 'Sponsored', 'SmartContractAddress', 'Id', 'BuiltOn', 'FullName', 'ImageUrl', 'ContentCreatedOn', 'PreMinedValue'])
trimmed_df.head()

,Algorithm,BlockNumber,BlockReward,BlockTime,CoinName,FullyPremined,IsTrading,Name,NetHashesPerSecond,ProofType,Symbol,TotalCoinSupply,TotalCoinsMined
42,Scrypt,147668,0,0,42 Coin,0,True,42,0,PoW/PoS,42,42,42
300,N/A,0,0,0,300 token,0,True,300,0,N/A,300,300,300
365,X11,NaN,NaN,NaN,365Coin,0,True,365,NaN,PoW/PoS,365,2300000000,NaN
404,Scrypt,236517,0,60,404Coin,0,True,404,0,PoW/PoS,404,532000000,0
433,N/A,8837713,2,0,433 Token,0,False,433,180814126711492,N/A,433,1000000000,1.08349e+08


In [78]:
# Keep only cryptocurrencies that are on trading
trimmed_df = trimmed_df.loc[trimmed_df['IsTrading'] == True]

In [79]:
# Keep only cryptocurrencies with a working algorithm
trimmed_df = trimmed_df.loc[trimmed_df['Algorithm'] != 'N/A']

In [80]:
# Remove the "IsTrading" column
trimmed_df = trimmed_df.drop(columns='IsTrading')

In [81]:
# Removing rows with at least 1 null value
trimmed_df = trimmed_df.dropna()

In [82]:
# Removing rows with cryptocurrencies without coins mined
trimmed_df = trimmed_df.loc[trimmed_df['TotalCoinsMined'] != 0]

In [83]:
# Dropping rows where there are 'N/A' text values
trimmed_df = trimmed_df.loc[trimmed_df['TotalCoinSupply'] != 'N/A']
trimmed_df = trimmed_df.loc[trimmed_df['ProofType'] != 'N/A']

In [84]:
# Fetching the cryptocurrencies names prior to drop them from crypto_df
coin_names = trimmed_df['CoinName']

In [85]:
# Removig the cryptocurrency name since it's not going to be used on the clustering algorithm
crypto_df = trimmed_df.drop(columns='CoinName')

In [50]:
# Creating dummies variables for text features
encoder = LabelEncoder()
for col in crypto_df.columns:
    if crypto_df[col].dtype == 'object':
        encoder = LabelEncoder()
        encoder.fit(list(crypto_df[col].values))
        crypto_df[col] = encoder.transform(list(crypto_df[col].values))

In [51]:
# Standardizing data
crypto_df_scaled = StandardScaler().fit_transform(crypto_df)

In [52]:
#Check the newly scaled data
crypto_df_scaled

array([[ 0.39812506, -0.61438516, -0.56873399, ..., -1.71574452,
         0.82785616, -1.74714646],
       [ 1.2085472 ,  1.54613483, -0.56873399, ..., -1.72879201,
         0.46810526,  1.57024521],
       [ 0.39812506,  1.39518147,  1.30207061, ...,  0.01957123,
         1.60731645,  0.44206343],
       ...,
       [-1.97525408, -0.08604839,  1.6263434 , ..., -1.41565232,
         1.00773161, -0.60553394],
       [ 0.05080128,  0.53663423,  1.30207061, ..., -1.33084366,
        -0.20342976, -0.28991166],
       [-1.85947949, -0.90685729, -0.56873399, ..., -1.57874591,
        -1.00687345,  0.66367056]])

### Reducing Dimensions Using PCA

In [53]:
# Using PCA to reduce dimension to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_df_scaled)

In [54]:
# Creating a DataFrame with the principal components data
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index
)
pca_df.head(10)

,PC 1,PC 2,PC 3
42,-2.053465,-1.973998,-1.352444
1337,-2.472462,-0.308549,0.064199
LTC,-0.978142,2.905481,0.037543
XMR,1.888935,2.176805,0.612703
NXT,1.409874,-2.357619,2.748224
ETC,-1.272211,0.770773,1.287831
ZEC,1.779584,2.480312,-0.029758
BTS,-0.697920,-3.561018,3.011969
DGB,-2.159885,2.265194,1.595408
BTCD,-1.440098,-1.874728,-0.999963


In [55]:
#Check the PCA variance
print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.sum())

[0.18565421 0.16091679 0.12195877]
0.46852976563931437


 _PCA features only explain ~47% of the Data_

### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [106]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range ok k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve", xlabel="Number of Clusters(k)", ylabel="Inertia")


:Curve   [k]   (inertia)

Running K-Means with `k=<you best value for k here>`

In [57]:
# Initialize the K-Means model
model = KMeans(n_clusters=6, random_state=0)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)

In [94]:
# Creating a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.DataFrame({
    "Algorithm": trimmed_df.Algorithm,
    "ProofType": trimmed_df.ProofType,
    "TotalCoinsMined": trimmed_df.TotalCoinsMined,
    "TotalCoinSupply": trimmed_df.TotalCoinSupply,
    "PC 1": pca_df['PC 1'],
    "PC 2": pca_df['PC 2'],
    "PC 3": pca_df['PC 3'],
    "CoinName": trimmed_df.CoinName,
    "Class": model.labels_,
    "Predictions": predictions, 
    },
    index=crypto_df.index
)
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class,Predictions
42,Scrypt,PoW/PoS,42,42,-2.053465,-1.973998,-1.352444,42 Coin,4,4
1337,X13,PoW/PoS,2.9312e+10,314159265359,-2.472462,-0.308549,0.064199,EliteCoin,4,4
LTC,Scrypt,PoW,6.30392e+07,84000000,-0.978142,2.905481,0.037543,Litecoin,0,0
XMR,CryptoNight-V7,PoW,1.7283e+07,0,1.888935,2.176805,0.612703,Monero,3,3
NXT,PoS,PoS/LPoS,1000000000,1000000000,1.409874,-2.357619,2.748224,Nxt,5,5
ETC,Ethash,PoW,114666216,210000000,-1.272211,0.770773,1.287831,Ethereum Classic,0,0
ZEC,Equihash,PoW,7.73819e+06,21000000,1.779584,2.480312,-0.029758,ZCash,3,3
BTS,SHA-512,PoS,2747160000,3600570502,-0.697920,-3.561018,3.011969,Bitshares,5,5
DGB,Multiple,PoW,1.24323e+10,21000000000,-2.159885,2.265194,1.595408,DigiByte,0,0
BTCD,SHA-256,PoW/PoS,1288862,22000000,-1.440098,-1.874728,-0.999963,BitcoinDark,4,4


### Visualizing Results

#### 3D-Scatter with Clusters

In [95]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_name='CoinName',
    hover_data= ['Algorithm'],
    color="Class",
    symbol="Class",
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [96]:
# Table with tradable cryptos
columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']
clustered_df.hvplot.table(columns)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

#### Scatter Plot with Tradable Cryptocurrencies

In [101]:
# Scaling data to create the scatter plot
clustered_df['TotalCoinsMined'] = clustered_df['TotalCoinsMined'].astype(float) / 1000000000
clustered_df['TotalCoinSupply'] = clustered_df['TotalCoinSupply'].astype(float) / 1000000000

In [102]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot(
    kind="scatter", 
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    c='Class', 
    colormap="viridis", 
    hover_cols=['CoinName']
)

:Scatter   [TotalCoinsMined]   (TotalCoinSupply,Class,CoinName)